In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import math
import numpy as np 
from nltk.corpus import stopwords 
from sklearn.model_selection import train_test_split 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Embedding, LSTM, Dense 
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.models import load_model 
import re
from keras.layers import InputLayer,Conv1D,Dense, Flatten, MaxPooling1D, RepeatVector
from sklearn.linear_model import LogisticRegression
import nltk
#from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
import math
import pickle
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score,recall_score,accuracy_score,f1_score,confusion_matrix,roc_auc_score
from sklearn.linear_model import Ridge

In [2]:
train = pd.read_csv('drugsComTrain_raw.csv')
test = pd.read_csv('drugsComTest_raw.csv')
print("Shape of train :", train.shape)
print("Shape of test :", test.shape)
#total size = 215063

Shape of train : (161297, 7)
Shape of test : (53766, 7)


In [3]:
data = pd.concat([train, test])
data.columns = ['ID','drug name','condition','review','rating','date','useful count']

In [4]:
df = data[data['useful count'] > 10]  

In [5]:
df_condition = df.groupby(['condition'])['drug name'].nunique().sort_values(ascending=False)
df_condition = pd.DataFrame(df_condition).reset_index()
df_condition.tail(20)

,condition,drug name
706,64</span> users found this comment helpful.,1
707,92</span> users found this comment helpful.,1
708,Gastritis/Duodenitis,1
709,Esophageal Variceal Hemorrhage Prophylaxis,1
710,98</span> users found this comment helpful.,1
711,Severe Mood Dysregulation,1
712,Short Stature,1
713,Short Stature for Age,1
714,Meningitis,1
715,"Skin Disinfection, Preoperative",1


In [6]:
# removing the conditions with <span> in it.

all_list = set(data.index)
span_list = []
for i,j in enumerate(data['condition']):
    if "</span>" in str(j):
        span_list.append(i)
new_idx = all_list.difference(set(span_list))
data = data.loc[list(new_idx)].reset_index()
del data['index']

In [7]:
df_condition = data.groupby(['condition'])['drug name'].nunique().sort_values(ascending=False)
df_condition = pd.DataFrame(df_condition).reset_index()
df_condition

,condition,drug name
0,Not Listed / Othe,253
1,Pain,219
2,Birth Control,181
3,High Blood Pressure,146
4,Acne,127
...,...,...
875,Legionella Pneumonia,1
876,Leukocytoclastic Vasculitis,1
877,Lewy Body Dementia,1
878,Pe,1


In [8]:
data=data.dropna()

In [9]:
data['sentiment'] = np.where(data['rating'] >= 6, 1, 0)

In [10]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
import re
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to C:\Users\Haripriya
[nltk_data]     K\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
english_stops = set(stopwords.words('english'))

In [12]:
stemmer = SnowballStemmer('english')
def review_to_words(raw_review):
    # 1. Delete HTML 
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 2. Make a space
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    # 3. lower letters
    words = letters_only.lower().split()
    # 5. Stopwords 
    meaningful_words = [w for w in words if not w in english_stops]
    # 6. Stemming
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    # 7. space join words
    return( ' '.join(stemming_words))

In [13]:
data['review_clean'] = data['review'].apply(review_to_words)
data.head()

C:\Users\Haripriya K\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


,ID,drug name,condition,review,rating,date,useful count,sentiment,review_clean
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,1,side effect take combin bystol mg fish oil
1,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22,1,tri antidepress year citalopram fluoxetin amit...
2,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,1,son halfway fourth week intuniv becam concern ...
3,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17-May-09,17,1,son crohn diseas done well asacol complaint sh...
4,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,0,use take anoth oral contracept pill cycl happi...


In [14]:
x_data=data['review_clean']
y_data=data['sentiment']

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3)
print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test,'\n')

Train Set
59392     love mirena month experienc littl bit spot fir...
169702    implanon year half say worst birth control per...
195178    use get yi time swim high school doctor would ...
148574    start pound take birth control month gain almo...
161431    say depress never felt low howev notic becom i...
                                ...                        
17745     pill year treat pcos period irregular experien...
168241    suffer chronic constip main due side effect me...
11658     alway get cold sore everi time get sick morn w...
107257    procedur feel like pap smear intens cramp noth...
156519    benzodiazepin year list work best treatment ge...
Name: review_clean, Length: 148865, dtype: object 

209868    realli good friend mine hook one morn accident...
194040    medicin suppos curv depress spike mania doctor...
31234     take year mix period year doctor took depress ...
202823    yeah seven day got hospit full bodi rash inclu...
9476                      high anxieti

In [16]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))
    return int(np.ceil(np.mean(review_length)))

In [17]:
token = Tokenizer(lower=False) # no need lower, because already lowered the data in loa
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)
max_length = get_max_length()
x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')
total_words = len(token.word_index) + 1 # add 1 because of 0 padding
print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[ 106  338    3 ...    0    0    0]
 [ 460    4  178 ...    0    0    0]
 [  18    8 1263 ...    0    0    0]
 ...
 [ 126    8  319 ...   17   25    1]
 [ 549   12   16 ...  837   95   82]
 [1874    4  836 ...    0    0    0]] 

Encoded X Test
 [[   40    54   447 ...     0     0     0]
 [   32   462  3435 ...     0     0     0]
 [    2     4   740 ...   504   195    31]
 ...
 [    9     2 30724 ...     2  1521   108]
 [  123    43    89 ...    28   381   324]
 [   18  1811     4 ...  1811  1054   129]] 

Maximum review length:  41


In [34]:
from sklearn.tree import DecisionTreeClassifier
clf_model = DecisionTreeClassifier(criterion="gini", random_state=42,max_depth=5, min_samples_leaf=5)   
clf_model.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=5, min_samples_leaf=5, random_state=42)

In [36]:
y_predict = clf_model.predict(x_test)
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(y_test,y_predict)

0.7057680250783699